In [6]:
import rospy
from IPython.display import display
try:
    rospy.get_node_uri()
except:
    rospy.init_node('robot_target_node', anonymous=True, disable_signals=True)

In [7]:
import rospy
import math
import numpy as np
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from rt1_assignment2_1.msg import Odom_simplified
from assignment_2_2024.msg import PlanningAction, PlanningGoal
import actionlib
from actionlib_msgs.msg import GoalStatus
import ipywidgets as widgets
from IPython.display import display, clear_output

class RobotTarget:
    def __init__(self):
        # Publishers and Subscribers
        self.odom_sub = rospy.Subscriber('/odom', Odometry, self.odom_callback)
        self.scan_sub = rospy.Subscriber('/scan', LaserScan, self.scan_callback)
        self.odom_simplified_pub = rospy.Publisher('/odom_simplified', Odom_simplified, queue_size=10)
        self.ac = actionlib.SimpleActionClient('reaching_goal', PlanningAction)

        # Wait for action server
        rospy.loginfo("Waiting for action server...")
        self.ac.wait_for_server()
        rospy.loginfo("Action server ready.")

        # Widgets for state display
        self.x_widget = widgets.FloatText(description='X:', value=0.0, disabled=True)
        self.y_widget = widgets.FloatText(description='Y:', value=0.0, disabled=True)
        self.dist_widget = widgets.FloatText(description='Closest Obs:', value=0.0, disabled=True)
        
        # Widgets for goal setting
        self.goal_x = widgets.FloatText(description='Goal X:')
        self.goal_y = widgets.FloatText(description='Goal Y:')
        self.send_button = widgets.Button(description='Send Goal', button_style='success')
        self.cancel_button = widgets.Button(description='Cancel Goal', button_style='danger')

        self.send_button.on_click(self.send_goal)
        self.cancel_button.on_click(self.cancel_goal)

        # Display all widgets
        display(widgets.HBox([self.x_widget, self.y_widget, self.dist_widget]))
        display(widgets.HBox([self.goal_x, self.goal_y, self.send_button, self.cancel_button]))

    def odom_callback(self, msg):
        x = msg.pose.pose.position.x
        y = msg.pose.pose.position.y

        self.x_widget.value = round(x, 2)
        self.y_widget.value = round(y, 2)

        # Publish simplified odometry
        simplified = Odom_simplified()
        simplified.x = x
        simplified.y = y
        simplified.vel_x = msg.twist.twist.linear.x
        simplified.vel_z = msg.twist.twist.angular.z
        self.odom_simplified_pub.publish(simplified)

    def scan_callback(self, msg):
        ranges = np.array(msg.ranges)
        valid_ranges = ranges[np.isfinite(ranges)]
        if valid_ranges.size > 0:
            min_dist = np.min(valid_ranges)
            self.dist_widget.value = round(min_dist, 2)
        else:
            self.dist_widget.value = float('inf')

    def send_goal(self, b=None):
        x = self.goal_x.value
        y = self.goal_y.value

        goal = PlanningGoal()
        goal.target_pose.header.stamp = rospy.Time.now()
        goal.target_pose.header.frame_id = "odom"
        goal.target_pose.pose.position.x = x
        goal.target_pose.pose.position.y = y

        rospy.loginfo(f"Sending goal to x={x}, y={y}")
        self.ac.send_goal(goal, done_cb=self.goal_done_cb)

    def cancel_goal(self, b=None):
        rospy.loginfo("Cancelling current goal...")
        self.ac.cancel_goal()

    def goal_done_cb(self, state, result=None):
        if state == GoalStatus.SUCCEEDED:
            rospy.loginfo("Goal reached!")
        else:
            rospy.logwarn(f"Goal did not succeed. State: {state}")


In [8]:
robot = RobotTarget()

[INFO] [1746048682.306388, 1807.892000]: Waiting for action server...
[INFO] [1746048682.308643, 1807.892000]: Action server ready.


[INFO] [1746048694.811427, 1820.193000]: Sending goal to x=1.0, y=1.0
[INFO] [1746048701.753442, 1827.014000]: Goal reached!
